In [1]:
%load_ext lab_black
%load_ext autoreload
%autoreload 1

### Imports

Use `aimport` for `schedview` imports for ease of debugging.

In [2]:
import warnings
import math
import os
import sys
import logging
from pathlib import Path
import panel as pn
import numpy as np
import pandas as pd
import param
import bokeh
from copy import deepcopy
import datetime
from pytz import timezone
import lzma
import pickle
import yaml
import json
import socket
import time
from contextlib import redirect_stdout
from pathlib import Path
from collections import OrderedDict
from tempfile import TemporaryDirectory, NamedTemporaryFile
import hashlib
import shutil

import sys
from conda.exceptions import EnvironmentLocationNotFound
from conda.gateways.disk.test import is_conda_environment
from conda.cli.main_list import print_packages

In [3]:
from astropy.time import Time, TimeDelta
from zoneinfo import ZoneInfo
import matplotlib as mpl
import matplotlib.pyplot as plt
import hvplot.pandas

In [4]:
import lsst.resources

In [5]:
import rubin_scheduler
from rubin_scheduler.scheduler.example import example_scheduler
from rubin_scheduler.scheduler import sim_runner
from rubin_scheduler.scheduler.model_observatory import ModelObservatory
from rubin_scheduler.scheduler.utils import SchemaConverter
from rubin_scheduler.sim_archive import sim_archive, drive_sim

In [6]:
import schedview
import schedview.app.prenight
import schedview.compute.scheduler
import schedview.collect.opsim
from schedview.plot.visitmap import BAND_COLORS

### Further preparation of the notebook

In [7]:
pn.extension("terminal")

### Filter warnings

Several dependencies throw prodigious instances of (benign) warnings.
Suppress them to avoid poluting the executed notebook.

In [8]:
warnings.filterwarnings(
    "ignore",
    module="astropy.time",
    message="Numerical value without unit or explicit format passed to TimeDelta, assuming days",
)
warnings.filterwarnings(
    "ignore",
    module="pandas",
    message="In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.",
)
warnings.filterwarnings(
    "ignore",
    module="healpy",
    message="divide by zero encountered in divide",
)
warnings.filterwarnings(
    "ignore",
    module="healpy",
    message="invalid value encountered in multiply",
)
warnings.filterwarnings(
    "ignore",
    module="holoviews",
    message="Discarding nonzero nanoseconds in conversion.",
)
warnings.filterwarnings(
    "ignore",
    module="rubin_sim",
    message="invalid value encountered in arcsin",
)
warnings.filterwarnings(
    "ignore",
    module="rubin_sim",
    message="All-NaN slice encountered",
)
warnings.filterwarnings(
    "ignore",
    module="rubin_sim.scheduler.utils",
    message="invalid value encountered in cast",
)
warnings.filterwarnings(
    "ignore",
    module="rubin_scheduler.scheduler.utils",
    message="invalid value encountered in cast",
)
warnings.filterwarnings(
    "ignore",
    module="rubin_scheduler.scheduler.surveys",
    message="All-NaN slice encountered",
)

## Configuration

Setting `keep_rewards` to `True` stores rewards that can be plotted in the schedview dashboards, but makes the simulation slower.

In [9]:
keep_rewards = True

Set the date we are simulating:

In [10]:
evening_iso8601 = "2025-05-05"

If we just use this day as the start and make the simulation duration 1 day, the begin and end of the simulation will probably begin in the middle on one night and end in the middle of the next.
Instead, find the sunset and sunrise of the night we want using the almanac, and use these to determine our start time and duration.

In [11]:
observatory = ModelObservatory()
night_sunset_info = observatory.almanac.get_sunset_info(
    evening_date=evening_iso8601, longitude=observatory.location.lon
)

mjd_start = night_sunset_info["sun_n12_setting"]
night_duration = night_sunset_info["sunrise"] - mjd_start

Now instantiate the `ModelObservatory` and scheduler we will actually use for the simulation.

In [12]:
observatory = ModelObservatory(mjd_start=mjd_start)

In [13]:
scheduler = example_scheduler(mjd_start=mjd_start)
scheduler.keep_rewards = True

INFO:healpy:Sigma is 254.796540 arcmin (0.074117 rad) 
INFO:healpy:-> fwhm is 600.000000 arcmin
INFO:healpy:Sigma is 0.000000 arcmin (0.000000 rad) 
INFO:healpy:-> fwhm is 0.000000 arcmin


Optimizing ELAISS1
Optimizing XMM_LSS
Optimizing ECDFS
Optimizing COSMOS
Optimizing EDFS_a


## Get a URI for the archive into which to save the simulation

For this sample, use a temporary directory:

In [14]:
if True:
    archive_dir = TemporaryDirectory()
    archive_uri = lsst.resources.ResourcePath(
        archive_dir.name, forceDirectory=True
    ).geturl()

If you have a local non-temporary you want to use instead, do something like this:

In [15]:
if False:
    archive_uri = "file:///my/directory/"

To save into the S3 bucket used by `schedview`, set both the uri and the environment variables necessary for access to it:

In [16]:
if False:
    os.environ["S3_ENDPOINT_URL"] = "https://s3dfrgw.slac.stanford.edu/"
    os.environ["AWS_PROFILE"] = "prenight"
    archive_uri = "s3://rubin-scheduler-prenight/opsim/"

In both of the above cases, be sure to include the trailing `/`.

## Save the notebook in a temporary directory

In [17]:
scratch_dir = TemporaryDirectory()
scratch_path = Path(scratch_dir.name)
notebook_fname = scratch_path.joinpath("notebook.ipynb").as_posix()

In [18]:
%notebook $notebook_fname

## Run the simulation

In [19]:
exec_start_time = Time.now()
results = drive_sim(
    observatory=observatory,
    scheduler=scheduler,
    archive_uri=archive_uri,
    label=f"Notebook test on {exec_start_time.iso}",
    notebook=notebook_fname,
    tags=["notebook", "devel"],
    mjd_start=mjd_start,
    survey_length=night_duration,
)

/sdf/data/rubin/user/neilsen/devel/rubin_scheduler/rubin_scheduler/skybrightness_pre/sky_model_pre.py:280: UserWarning: Requested MJD between sunrise and sunset, returning closest maps
  warnings.warn("Requested MJD between sunrise and sunset, returning closest maps")
/sdf/data/rubin/user/neilsen/devel/rubin_scheduler/rubin_scheduler/skybrightness_pre/sky_model_pre.py:280: UserWarning: Requested MJD between sunrise and sunset, returning closest maps
  warnings.warn("Requested MJD between sunrise and sunset, returning closest maps")
/sdf/data/rubin/user/neilsen/devel/rubin_scheduler/rubin_scheduler/skybrightness_pre/sky_model_pre.py:280: UserWarning: Requested MJD between sunrise and sunset, returning closest maps
  warnings.warn("Requested MJD between sunrise and sunset, returning closest maps")
/sdf/data/rubin/user/neilsen/devel/rubin_scheduler/rubin_scheduler/skybrightness_pre/sky_model_pre.py:280: UserWarning: Requested MJD between sunrise and sunset, returning closest maps
  warnin

progress = 89.28%

/sdf/data/rubin/user/neilsen/devel/rubin_scheduler/rubin_scheduler/skybrightness_pre/sky_model_pre.py:280: UserWarning: Requested MJD between sunrise and sunset, returning closest maps
  warnings.warn("Requested MJD between sunrise and sunset, returning closest maps")
/sdf/data/rubin/user/neilsen/devel/rubin_scheduler/rubin_scheduler/skybrightness_pre/sky_model_pre.py:280: UserWarning: Requested MJD between sunrise and sunset, returning closest maps
  warnings.warn("Requested MJD between sunrise and sunset, returning closest maps")


progress = 193.88%

/sdf/data/rubin/user/neilsen/devel/rubin_scheduler/rubin_scheduler/skybrightness_pre/sky_model_pre.py:280: UserWarning: Requested MJD between sunrise and sunset, returning closest maps
  warnings.warn("Requested MJD between sunrise and sunset, returning closest maps")


Skipped 0 observations
Flushed 33 observations from queue for being stale
Completed 1023 observations
ran in 0 min = 0.0 hours


/sdf/data/rubin/user/neilsen/devel/rubin_scheduler/rubin_scheduler/sim_archive/sim_archive.py:108: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block3_values] [items->Index(['basis_function', 'tier_label', 'survey_label', 'survey_class',
       'basis_function_class'],
      dtype='object')]

  reward_df.to_hdf(rewards_fname, "reward_df")
INFO:rubin_scheduler.sim_archive.sim_archive:Copied /tmp/tmp100nb6vg/environment.txt to file:///tmp/tmps5gertro/2024-01-04/1/environment.txt
INFO:rubin_scheduler.sim_archive.sim_archive:Copied /tmp/tmp100nb6vg/notebook.ipynb to file:///tmp/tmps5gertro/2024-01-04/1/notebook.ipynb
INFO:rubin_scheduler.sim_archive.sim_archive:Copied /tmp/tmp100nb6vg/opsim.db to file:///tmp/tmps5gertro/2024-01-04/1/opsim.db
INFO:rubin_scheduler.sim_archive.sim_archive:Copied /tmp/tmp100nb6vg/pypi.json to file:///tmp/tmps5gertro/2024-01-04/1/pypi.json
INFO:rubin_sched

In [20]:
sim_uri = results[-1]
sim_uri

ResourcePath("file:///tmp/tmps5gertro/2024-01-04/1/")